In [1]:
import cv2
import numpy as np
print(cv2.__version__)

4.5.1


In [10]:
def get_ht(s, d):
    a = np.array([[s[0][0], s[0][1], 1, 0, 0, 0, -s[0][0] * d[0][0], -s[0][1] * d[0][0]],
         [s[1][0], s[1][1], 1, 0, 0, 0, -s[1][0] * d[1][0], -s[1][1] * d[1][0]],
         [s[2][0], s[2][1], 1, 0, 0, 0, -s[2][0] * d[2][0], -s[2][1] * d[2][0]],
         [s[3][0], s[3][1], 1, 0, 0, 0, -s[3][0] * d[3][0], -s[3][1] * d[3][0]],
         [0, 0, 0, s[0][0], s[0][1], 1, -s[0][0] * d[0][1], -s[0][1] * d[0][1]],
         [0, 0, 0, s[1][0], s[1][1], 1, -s[1][0] * d[1][1], -s[1][1] * d[1][1]],
         [0, 0, 0, s[2][0], s[2][1], 1, -s[2][0] * d[2][1], -s[2][1] * d[2][1]],
         [0, 0, 0, s[3][0], s[3][1], 1, -s[3][0] * d[3][1], -s[3][1] * d[3][1]]])
    
    b = np.array([d[0][0], d[1][0], d[2][0], d[3][0], d[0][1], d[1][1], d[2][1], d[3][1]])
    
    print('a.shape = ', a.shape)
    print('b.shape = ', b.shape)
    
    invA = np.linalg.inv(a)
    ht = np.dot(invA, b)
    return ht

In [12]:
def transform(h, s):
    x = (h[0]*s[0] + h[1]*s[1] + h[2])/(h[6]*s[0] + h[7]*s[1] + 1)
    y = (h[3]*s[0] + h[4]*s[1] + h[5])/(h[6]*s[0] + h[7]*s[1] + 1)
    
    return x, y

In [3]:
pixel_points = [[1019, 205],[378, 68],[262, 716],[950, 839]]
real_points = [[0, 0],[70, 0],[70, 70],[0, 70]]

ref_points = [[768, 806], [556, 762], [650, 462]]

In [14]:
h = get_ht(pixel_points, real_points)
print('h = ', h)
print('h.shape = ', h.shape)
print('h[0,0] =', h[0])

list_ref = []
for ref in ref_points:
    x,y = transform(ht_ori, ref)
    list_ref.append([x,y])
    print('pix {}, {} => real: x = {}, y = {}'.format(ref[0], ref[1],x,y))


a.shape =  (8, 8)
b.shape =  (8,)
h =  [-1.04531209e-01 -1.13764250e-02  1.08849469e+02 -2.32908171e-02
  1.08973823e-01  1.39370889e+00 -7.08758982e-05  9.21116530e-05]
h.shape =  (8,)
h[0,0] = -0.10453120902190115
pix 768, 806 => real: x = 19.023264464787882, y = 69.95353202390346
pix 556, 762 => real: x = 40.805211774533895, y = 69.34741017010587
pix 650, 462 => real: x = 35.77397572709498, y = 36.72964279773875


In [23]:
pix_p = np.float32([1019, 205, 378, 68, 262, 716, 950, 839]).reshape(-1,1,2)
real_p = np.float32([0, 0, 70, 0, 70, 70, 0, 70]).reshape(-1,1,2)
print(pix_p.shape)
print(real_p.shape)

ht, status = cv2.findHomography(pix_p, real_p,cv2.RHO)#cv2.RANSAC)
print(ht)
print(status)

(4, 1, 2)
(4, 1, 2)
[[-1.04531209e-01 -1.13764250e-02  1.08849469e+02]
 [-2.32908171e-02  1.08973823e-01  1.39370889e+00]
 [-7.08758982e-05  9.21116530e-05  1.00000000e+00]]
[[1]
 [1]
 [1]
 [1]]


In [20]:
# perspectiveTransform(vSrcPoint, vDstPoint, this->ht);
# (4, 1, 2)
# (4, 1, 2)
# [[-1.04531209e-01 -1.13764250e-02  1.08849469e+02]
#  [-2.32908171e-02  1.08973823e-01  1.39370889e+00]
#  [-7.08758982e-05  9.21116530e-05  1.00000000e+00]]
# [[1]
#  [1]
#  [1]
#  [1]]
pts = np.float32([ [768, 806], [556, 762], [650, 462] ]).reshape(-1,1,2)
dst = cv2.perspectiveTransform(pts,ht)
print(dst)
    

[[[19.023264 69.95353 ]]

 [[40.80521  69.34741 ]]

 [[35.773975 36.72964 ]]]


In [24]:
M = cv2.getPerspectiveTransform(pix_p, real_p)
print(M)

[[-1.04531209e-01 -1.13764250e-02  1.08849469e+02]
 [-2.32908171e-02  1.08973823e-01  1.39370889e+00]
 [-7.08758982e-05  9.21116530e-05  1.00000000e+00]]
